In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [2]:
# size_width_rugs_runners
# size_length_rugs_runners
# size_diameter_rugs_runners

# parameters
customer_id = '5'
formatted_attribute = 'metal_finish'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Curtain Hardware"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r"(?i)(\bbacking)|()"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/bbby.sql', params=params)
# df=df[df['value']=='n/a']
df

attribute      value    customers_name external_id  \
0      metal_finish    Antique  bedbathandbeyond    18935775   
1      metal_finish    Antique  bedbathandbeyond    40040158   
2      metal_finish    Antique  bedbathandbeyond    40040400   
3      metal_finish    Antique  bedbathandbeyond    40358543   
4      metal_finish    Antique  bedbathandbeyond    40361000   
...             ...        ...               ...         ...   
14445  metal_finish  Weathered  bedbathandbeyond    62834629   
14446  metal_finish  Weathered  bedbathandbeyond    64691428   
14447  metal_finish  Weathered  bedbathandbeyond    65780411   
14448  metal_finish  Weathered  bedbathandbeyond    69587536   
14449  metal_finish  Weathered  bedbathandbeyond    69587766   

                                                                   long_desc  \
0                                       Uttermost Karline Small Vanity Bench   
1                      ELK Lighting Buckingham Tiffany Ceiling Fan/Light Kit   
2      ELK Lighting Buckingham Tiffany 1-Light Semi-Flush in Vintage Antique   
3                                          Richelieu 3-Inch Traditional Pull   
4                                   Richelieu Beaded Knob in Antique English   
...                                                                      ...   
14445   Troy Lighting Menlo Park 4-Light Wall Mount Vanity Bar in Old Silver   
14446       Boston International Weathered Metal Shovel Wall Hanger in Brown   
14447  Danya B.™ 3-Tier Vertical Pipe Industrial Wall Shelf in Dark Chestnut   
14448         RiverRidge® Home Afton Weathered Wood 2-Pack 3-Hook Coat Racks   
14449               RiverRidge® Home Woodbury Shoe Cabinet in Weathered Wood   

      resolution  
0       standard  
1       standard  
2       standard  
3       standard  
4       standard  
...          ...  
14445   standard  
14446   standard  
14447   standard  
14448   standard  
14449   standard  

[14450 rows x 6 columns]

In [5]:
# error

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Antique',
 'Brass',
 'Bronze',
 'Brushed',
 'Chrome',
 'Copper',
 'Gold',
 'Gunmetal',
 'Matte',
 'Mirror',
 'Nickel',
 'Oil Rubbed',
 'Pewter',
 'Polished',
 'Powder Coat',
 'Powder Coated',
 'Satin',
 'Weathered']

In [7]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0                 []
1                 []
2          [Antique]
3                 []
4          [Antique]
            ...     
14445             []
14446    [Weathered]
14447             []
14448    [Weathered]
14449    [Weathered]
Name: long_desc, Length: 14450, dtype: object

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()

Brass         5356
Nickel        2453
Polished      1904
Chrome        1606
Bronze        1593
Satin         1414
Matte         1123
Oil Rubbed     988
Brushed        933
Mirror         741
Gold           219
Antique        146
Pewter         124
Copper          33
Weathered       10
Gunmetal         4
Name: matches, dtype: int64

In [9]:
# def re_extract(pattern, txt):
#     matches = re.findall(pattern, txt)
#     tmp_matches = []
#     for match in matches:
#         for tup in match:
#             if tup != '':
#                 tmp_matches.append(tup)
#     return list(set(tmp_matches))
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df

In [10]:
df=df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]

In [12]:
df

attribute value    customers_name external_id  \
4985   metal_finish   n/a  bedbathandbeyond    10592364   
4991   metal_finish   n/a  bedbathandbeyond    11170544   
5026   metal_finish   n/a  bedbathandbeyond    13436606   
5044   metal_finish   n/a  bedbathandbeyond    14050760   
5045   metal_finish   n/a  bedbathandbeyond    14050779   
...             ...   ...               ...         ...   
10279  metal_finish   n/a  bedbathandbeyond    69589691   
10374  metal_finish   n/a  bedbathandbeyond    69599212   
10480  metal_finish   n/a  bedbathandbeyond    69636430   
10498  metal_finish   n/a  bedbathandbeyond    69669737   
10515  metal_finish   n/a  bedbathandbeyond    69695080   

                                                                               long_desc  \
4985                           Zadro® Z'Fogless™ Adjustable Magnification Shaving Mirror   
4991                         Seville Classics® Mesh Metal 6-Gallon Wastebasket in Chrome   
5026                                        Gatco® Latitude II Robe Hook in Satin Nickel   
5044                                      India Ink Winston Tumbler in Oil Rubbed Bronze   
5045                            India Ink Winston Toothbrush Holder in Oil Rubbed Bronze   
...                                                                                  ...   
10279                                     ORG 2-Tier Metal Mesh Cabinet Drawer in Nickel   
10374          JONATHAN Y Jayne 4-Light Wall Mount LED Vanity Light in Oil Rubbed Bronze   
10480                                     Smart Clean Automatic Soap Dispenser in Chrome   
10498  J. Queen New York™ Decade 2-Pack 84-Inch Rod Pocket Window Curtain Panels in Gold   
10515                                        Deluxe Metal Oval Mirrored Bar Cart in Gold   

      resolution               matches  
4985    standard              [Mirror]  
4991    standard              [Chrome]  
5026    standard       [Satin, Nickel]  
5044    standard  [Oil Rubbed, Bronze]  
5045    standard  [Oil Rubbed, Bronze]  
...          ...                   ...  
10279   standard              [Nickel]  
10374   standard  [Oil Rubbed, Bronze]  
10480   standard              [Chrome]  
10498   standard                [Gold]  
10515   standard                [Gold]  

[509 rows x 7 columns]

In [ ]:
# df=df[(df['matches'].astype(str)!='[]')]
# df

In [13]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [14]:
na_to_gsheet(df)

succesfully saved to worksheet:`n/a` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


In [15]:
matches=df

In [16]:
matches[curation_col]=matches['matches']
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

<ipython-input-16-a4929d6a6f39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches[curation_col]=matches['matches']
<ipython-input-16-a4929d6a6f39>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['external_id']=matches['external_id'].apply('="{}"'.format)


attribute value    customers_name  external_id  \
4985   metal_finish   n/a  bedbathandbeyond  ="10592364"   
4991   metal_finish   n/a  bedbathandbeyond  ="11170544"   
5026   metal_finish   n/a  bedbathandbeyond  ="13436606"   
5044   metal_finish   n/a  bedbathandbeyond  ="14050760"   
5045   metal_finish   n/a  bedbathandbeyond  ="14050779"   
...             ...   ...               ...          ...   
10279  metal_finish   n/a  bedbathandbeyond  ="69589691"   
10374  metal_finish   n/a  bedbathandbeyond  ="69599212"   
10480  metal_finish   n/a  bedbathandbeyond  ="69636430"   
10498  metal_finish   n/a  bedbathandbeyond  ="69669737"   
10515  metal_finish   n/a  bedbathandbeyond  ="69695080"   

                                                                               long_desc  \
4985                           Zadro® Z'Fogless™ Adjustable Magnification Shaving Mirror   
4991                         Seville Classics® Mesh Metal 6-Gallon Wastebasket in Chrome   
5026                                        Gatco® Latitude II Robe Hook in Satin Nickel   
5044                                      India Ink Winston Tumbler in Oil Rubbed Bronze   
5045                            India Ink Winston Toothbrush Holder in Oil Rubbed Bronze   
...                                                                                  ...   
10279                                     ORG 2-Tier Metal Mesh Cabinet Drawer in Nickel   
10374          JONATHAN Y Jayne 4-Light Wall Mount LED Vanity Light in Oil Rubbed Bronze   
10480                                     Smart Clean Automatic Soap Dispenser in Chrome   
10498  J. Queen New York™ Decade 2-Pack 84-Inch Rod Pocket Window Curtain Panels in Gold   
10515                                        Deluxe Metal Oval Mirrored Bar Cart in Gold   

      resolution               matches        Q:metal_finish  
4985    standard              [Mirror]              [Mirror]  
4991    standard              [Chrome]              [Chrome]  
5026    standard       [Satin, Nickel]       [Satin, Nickel]  
5044    standard  [Oil Rubbed, Bronze]  [Oil Rubbed, Bronze]  
5045    standard  [Oil Rubbed, Bronze]  [Oil Rubbed, Bronze]  
...          ...                   ...                   ...  
10279   standard              [Nickel]              [Nickel]  
10374   standard  [Oil Rubbed, Bronze]  [Oil Rubbed, Bronze]  
10480   standard              [Chrome]              [Chrome]  
10498   standard                [Gold]                [Gold]  
10515   standard                [Gold]                [Gold]  

[509 rows x 8 columns]

In [ ]:
error----Switch Max and Min when doing length and width

In [17]:
gc = gcloud.Gsheets()

In [18]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [19]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [20]:
looks_good('Bed Bath & Beyond', attribute, df, matches)